In [1]:
from mistralai import Mistral
import yaml
import json
import requests 
from tqdm.auto import notebook_tqdm
import minsearch
# from elasticsearch import Elasticsearch

/home/codespace/.python/current/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
api_key = "ollama"
model = "mistral"

In [5]:
client = Mistral(server_url="http://127.0.0.1:11434", api_key=api_key)

In [6]:
chat_response = client.chat.complete(
    model=model,
    messages=[{"role":"user", "content":"What is the best French cheese?"}]
)

In [7]:
print(chat_response)

id='chatcmpl-451' object='chat.completion' model='mistral' usage=UsageInfo(prompt_tokens=12, completion_tokens=178, total_tokens=190) created=1739871416 choices=[ChatCompletionChoice(index=0, message=AssistantMessage(content=' The "best" French cheese can be subjective, as it depends on personal taste. However, Brie de Meaux and Roquefort are often cited as classic examples of excellent French cheeses. Brie de Meaux, a soft, creamy cheese from the Brie region nearby Paris, is known for its mild yet rich flavor and distinctive bloomy rind. Roquefort, on the other hand, is a blue-veined cheese made from sheep\'s milk in the town of Roquefort-sur-Soulzon. It has a strong flavor with saltiness balanced by a creamy texture. Some others to consider are Camembert de Normandie and Comté. Ultimately, there\'s no wrong answer when it comes to enjoying French cheese, as long as you savor the unique qualities of each variety!', tool_calls=Unset(), prefix=False, role='assistant'), finish_reason='st

Aqui puedo modificar la URL a la que apunta el codigo, en lugar de a la de mistral api, a la del puerto desde el que se puede acceder a ollama. No necesitaria una key porque mi puerto esta abierto sino que tendria que apuntar a la aplicacion ollama para que se ejecute. luego podria pedirle un modelo de ollama, en este caso podria usar mistral 7b en lugar de mistral small latest.


1. hay que hacer ollama serve para que se habra la url 
2. luego hay que hacer ollama pull para que descargue el modelo
3. a partir de ahi se pude usar

In [8]:
docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [9]:
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

In [15]:
index.fit(documents)

In [16]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )

    return results

In [17]:
def build_prompt(query, search_results):
    prompt_template = """
        You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
        Use only the facts from the CONTEXT when answering the QUESTION.

        QUESTION: {question}

        CONTEXT: 
        {context}
        """.strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [18]:
def llm(prompt):
    response = client.chat.complete(
        model='mistral',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [19]:
query = 'how do I run kafka?'

def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [20]:
rag(query)

' To run Kafka with Java, you can follow these steps:\n\n1. Navigate to the project directory.\n2. Run the command in the terminal (replace `<jar_name>` with your project\'s specific jar name):\n   ```\n   java -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java\n   ```\n\nFor Python, the methods to run Kafka depend on your operating system and specific requirements:\n\nIf you\'re using a Linux distribution or MacOS:\n1. Create a virtual environment using Python 3 (python3 instead of python):\n   ```\n   python3 -m venv env\n   source env/bin/activate\n   pip install -r requirements.txt\n   ```\n2. Activate the created virtual environment and run your Kafka-related Python scripts. You should also ensure that Docker images are running first.\n\nIf you encounter a **"ModuleNotFoundError: No module named \'kafka.vendor.six.moves\'"** error when using kafka-python, you can try using `kafka-python-ng` instead by installing with pip:\n```\npip install k